In [6]:
# Librerías principales
import pandas as pd
import numpy as np
import os


In [3]:
# Definir la ruta al archivo
ruta_dta = "C:/SEM 8/Bloque dos & materias final/Proyecto BeWay/tablas & codigos/dtas/BASE_ENSAFI.dta"

# Leer el archivo .dta
data = pd.read_stata(ruta_dta)

# Ver las primeras filas para asegurarnos
#print(data.head())


In [7]:
# Definir ruta de salida
output_path = os.path.join('..', 'outputspy', 'vars_list.txt')

# Extraer los nombres de las variables
variable_names = data.columns.tolist()

# Escribir (sobrescribir) el archivo txt
with open(output_path, 'w', encoding='utf-8') as f:
    for var in variable_names:
        f.write(f"{var}\n")

print(f"Archivo 'vars_list.txt' creado exitosamente con {len(variable_names)} variables.")


Archivo 'vars_list.txt' creado exitosamente con 336 variables.


In [11]:


# Paso 1: Crear copia del dataframe original
data_cleaned = data.copy()

# Paso 2: Limpiar y convertir todo a numérico
data_cleaned = data_cleaned.applymap(lambda x: pd.to_numeric(str(x).strip(), errors='coerce'))

# Paso 3: Detectar columnas con valores exclusivamente 1 o 2
cols_si_no = []
for col in data_cleaned.columns:
    unique_vals = data_cleaned[col].dropna().unique()
    if set(unique_vals).issubset({1, 2}):
        cols_si_no.append(col)

# Paso 4: Transformar 2 → 0
data_cleaned[cols_si_no] = data_cleaned[cols_si_no].replace({2: 0})

# Paso 5: Convertir a int si no hay NaN
for col in cols_si_no:
    if data_cleaned[col].isna().sum() == 0:
        data_cleaned[col] = data_cleaned[col].astype(int)

# Paso 6: Crear tabla compacta
records = []
for col in data_cleaned.columns:
    status = 'modificada' if col in cols_si_no else 'no_modificada'
    valores_unicos = data_cleaned[col].dropna().unique()
    n_nas = data_cleaned[col].isna().sum()
    
    # Formateo compacto:
    if len(valores_unicos) <= 5:
        valores_str = '(' + ', '.join(map(str, sorted(valores_unicos))) + ')'
    else:
        valores_str = f"min={np.min(valores_unicos)}, max={np.max(valores_unicos)}, únicos={len(valores_unicos)}"
    
    records.append({
        'variable_name': col,
        'status': status,
        'valores': valores_str,
        'n_nas': n_nas
    })

status_df = pd.DataFrame.from_records(records)

# Paso 7: Guardar el archivo
output_path = os.path.join('..', 'outputspy', 'vars_list.txt')
status_df.to_csv(output_path, sep='\t', index=False)

print(f"Archivo 'vars_list.txt' actualizado con {len(status_df)} variables.")
print(status_df.head())



C:\Users\andre\AppData\Local\Temp\ipykernel_9312\2404438184.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_cleaned = data_cleaned.applymap(lambda x: pd.to_numeric(str(x).strip(), errors='coerce'))


Archivo 'vars_list.txt' actualizado con 336 variables.
  variable_name         status                                    valores  \
0      llaveviv  no_modificada  min=10001601, max=326080020, únicos=20201   
1          p1_1  no_modificada                     min=1, max=9, únicos=9   
2          p1_2  no_modificada                   min=1, max=18, únicos=16   
3          p1_3  no_modificada                     min=0, max=8, únicos=9   
4       p1_4_01     modificada                                     (0, 1)   

   n_nas  
0      0  
1      0  
2      0  
3      0  
4      0  


In [12]:
#MONITOREO DE DF


# Paso 1: Definir la ruta de entrada (vars_list.txt)
input_path = os.path.join('..', 'outputspy', 'vars_list.txt')

# Leer el archivo
vars_df = pd.read_csv(input_path, sep='\t')

# Paso 2: Filtrar variables que tienen status == 'modificada'
modificadas_df = vars_df[vars_df['status'] == 'modificada']

# Paso 3: Definir la ruta de salida (monitoreo.txt)
output_path = os.path.join('..', 'outputspy', 'monitoreo.txt')

# Guardar solo las modificadas
modificadas_df.to_csv(output_path, sep='\t', index=False)

print(f"Archivo 'monitoreo.txt' creado con {len(modificadas_df)} variables modificadas.")
print(modificadas_df.head())


Archivo 'monitoreo.txt' creado con 125 variables modificadas.
  variable_name      status valores  n_nas
4       p1_4_01  modificada  (0, 1)      0
5       p1_4_02  modificada  (0, 1)      0
6       p1_4_03  modificada  (0, 1)      0
7       p1_4_04  modificada  (0, 1)      0
8       p1_4_05  modificada  (0, 1)      0
